# Helper Functions and Packages

In [52]:
from bitarray import bitarray
from bitarray.util import ba2int
import numpy as np

In [54]:
def write2testfile(f, *args):
    for arg in args:
        f.write(arg)
        f.write(' ')
    f.write('\n')

# ALU Test Cases

In [53]:
def sra(x,n,m):
    if x & 2**(n-1) != 0:  # MSB is 1, i.e. x is negative
        filler = int('1'*m + '0'*(n-m),2)
        x = (x >> m) | filler  # fill in 0's with 1's
        return x
    else:
        return x >> m

In [55]:
def ALU(operand1, operand2, funct7, funct3):
    '''
    operand1 and operand2 are both bitarray.bitarray()
    '''
    funct = funct7+funct3
    if funct == '0000000000':
        temp_op1 = np.array(ba2int(operand1, signed = False), dtype='uint32')
        temp_op2 = np.array(ba2int(operand2, signed = False), dtype='uint32')
        result = temp_op1 + temp_op2
        return bin(result)[2:].zfill(32)
    elif funct == '0100000000':
        temp_op1 = np.array(ba2int(operand1, signed = False), dtype='uint32')

        temp_op2 = np.array(ba2int(operand2, signed = False), dtype='uint32')

        result = temp_op1 - temp_op2
        return bin(result)[2:].zfill(32)
    elif funct == '0000000001':
        result = operand1 << ba2int(operand2[-5:], signed = False)
        return result.to01()
    elif funct == '0000000010':
        temp_op1 = ba2int(operand1, signed = True)
        temp_op2 = ba2int(operand2, signed = True)
        if temp_op1 < temp_op2:
            return bin(1)[2:].zfill(32)
        else:
            return bin(0)[2:].zfill(32)
    elif funct == '0000000011':
        temp_op1 = ba2int(operand1, signed = False)
        temp_op2 = ba2int(operand2, signed = False)
        if temp_op1 < temp_op2:
            return bin(1)[2:].zfill(32)
        else:
            return bin(0)[2:].zfill(32)
    elif funct == '0000000100':
        result = operand1 ^ operand2
        return result.to01()
    elif funct == '0000000101':
        result = operand1 >> ba2int(operand2[-5:], signed = False)
        return result.to01()
    elif funct == '0100000101':
        temp_op2 = ba2int(operand2[-5:], signed = False)
        result = bin(sra(int(operand1.to01(), 2), 32, temp_op2))[2:].zfill(32)
        return result
    elif funct == '0000000110':
        result = operand1 | operand2
        return result.to01()
    elif funct == '0000000111':
        result = operand1 & operand2
        return result.to01()

In [56]:
possible_arithmetic = ['0000000000', '0100000000', '0000000001', '0000000010', '0000000011', '0000000100', '0000000101', 
                       '0100000101', '0000000110', '0000000111']


with open('ALU_tcases.txt', 'w') as f:
    for opcode in possible_arithmetic:
        for _ in range(30):
            operand1 = np.random.randint(0, 4294967296, dtype='uint32')
            operand2 = np.random.randint(0, 4294967296, dtype='uint32')
            operand1 = bitarray(bin(operand1)[2:].zfill(32))
            operand2 = bitarray(bin(operand2)[2:].zfill(32))
            funct7 = opcode[:7]
            funct3 = opcode[7:]
            out = ALU(operand1, operand2, funct7, funct3)
            write2testfile(f, operand1.to01(), operand2.to01(), funct7, funct3, out)


# Program Counter Test Cases

In [64]:
with open('PC_tcases.txt', 'w') as f:
    # simply adding 4 to the address
    address = 0x01000000
    for t in range(30):
        write2testfile(f, '0', hex(address)[2:].zfill(8), hex(address)[2:].zfill(8))
        address += 0x4

    # randomly pressing reset
    address = 0x01000000
    for t in range(100):
        rst = np.random.randint(0, 2, size=None)
        if rst == 1:
            address = 0x01000000
            write2testfile(f, '1', hex(address)[2:].zfill(8), hex(address)[2:].zfill(8))
        else:
            write2testfile(f, '0', hex(address)[2:].zfill(8), hex(address)[2:].zfill(8))
            address += 0x4

# Register File Test Cases

In [68]:
with open('reg_tcases.txt', 'w') as f:
    # First we write a lot of stuff into the reg file
    for i in range(1, 32):
        rd = bin(i)[2:].zfill(5)
        r1 = '00001'
        r2 = '00001'
        write_data = bin(i)[2:].zfill(32)
        w_en = '1'
        r_en = '0'
        r1_read = ''.zfill(32)
        r2_read = ''.zfill(32)
        write2testfile(f, rd, r1, r2, write_data, w_en, r_en, r1_read, r2_read)

    # Then we read all the register except r0
    for i in range(1, 16):
        rd = ''.zfill(5)
        r1 = bin(i)[2:].zfill(5)
        r2 = bin(2*i)[2:].zfill(5)
        write_data = bin(i)[2:].zfill(32)
        w_en = '0'
        r_en = '1'
        r1_read = bin(i)[2:].zfill(32)
        r2_read = bin(i*2)[2:].zfill(32)
        write2testfile(f, rd, r1, r2, write_data, w_en, r_en, r1_read, r2_read)

    # Tried write to 0 and read 0
    write2testfile(f, '00000', '00000', '00000', '11111'.zfill(32), '1', '1', ''.zfill(32), ''.zfill(32))
    write2testfile(f, '00000', '00000', '00000', '11111'.zfill(32), '1', '1', ''.zfill(32), ''.zfill(32))